In [1]:
import pandas as pd
import numpy as np

df_a = pd.read_csv('properties_2016.csv')
df_b = pd.read_csv('properties_2017.csv')

/Users/bennett/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/bennett/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Training Data

In [2]:
print(f'FIPS Coverage: {df_a.censustractandblock.notna().sum()/len(df_a):.3f}')

# Drop Records w/o FIPS values
df_a = df_a[df_a.censustractandblock.notna()] 

# Recode Features with Boolean Values
bool_cols = ['hashottuborspa','fireplaceflag'] 

df_a.loc[:,bool_cols] = (df_a.loc[:,bool_cols] == True).astype(int)
    
# Code Zeros for "If Any" Features
if_any_cols = ['fireplacecnt','poolcnt']

df_a.loc[:, if_any_cols] = df_a.loc[:, if_any_cols].fillna(0) 
    
non_actionable_cols = [
    # Geographic
    'latitude',
    'longitude',
    'rawcensustractandblock', # Duplicative w/ Processed Values
    'regionidzip',
    'regionidcounty', # Duplicative w/ FIPs
    
]

actionable_cols = [i for i in df_a.columns if i not in non_actionable_cols]

df_a = df_a.loc[:, actionable_cols]

FIPS Coverage: 0.975


In [3]:
# Columns by Null Records
df_a.isna().sum().sort_values()/len(df_a)

parcelid                        0.000000e+00
assessmentyear                  0.000000e+00
fireplaceflag                   0.000000e+00
propertylandusetypeid           0.000000e+00
poolcnt                         0.000000e+00
hashottuborspa                  0.000000e+00
fireplacecnt                    0.000000e+00
fips                            0.000000e+00
censustractandblock             0.000000e+00
bathroomcnt                     0.000000e+00
bedroomcnt                      0.000000e+00
roomcnt                         3.436319e-07
propertycountylandusecode       4.810846e-06
taxvaluedollarcnt               1.539814e-03
structuretaxvaluedollarcnt      3.839055e-03
taxamount                       4.558620e-03
landtaxvaluedollarcnt           9.674268e-03
calculatedfinishedsquarefeet    1.100962e-02
yearbuilt                       1.233432e-02
regionidcity                    1.611187e-02
calculatedbathnbr               2.237387e-02
fullbathcnt                     2.237387e-02
lotsizesqu

Drop Columns with >= 40% missingness (per https://www.sciencedirect.com/science/article/pii/S0895435618308710#:~:text=Statistical%20guidance%20articles%20have%20stated,18%5D%2C%20%5B19%5D.)

In [4]:
keep_cols = df_a.isna().sum().sort_values()/len(df_a) < .6
df_a = df_a.loc[:, keep_cols]

df_a.isna().sum().sort_values()/len(df_a)

parcelid                        0.000000e+00
assessmentyear                  0.000000e+00
fireplaceflag                   0.000000e+00
propertylandusetypeid           0.000000e+00
hashottuborspa                  0.000000e+00
fireplacecnt                    0.000000e+00
fips                            0.000000e+00
poolcnt                         0.000000e+00
censustractandblock             0.000000e+00
bathroomcnt                     0.000000e+00
bedroomcnt                      0.000000e+00
roomcnt                         3.436319e-07
propertycountylandusecode       4.810846e-06
taxvaluedollarcnt               1.539814e-03
structuretaxvaluedollarcnt      3.839055e-03
taxamount                       4.558620e-03
landtaxvaluedollarcnt           9.674268e-03
calculatedfinishedsquarefeet    1.100962e-02
yearbuilt                       1.233432e-02
regionidcity                    1.611187e-02
calculatedbathnbr               2.237387e-02
fullbathcnt                     2.237387e-02
lotsizesqu

In [5]:
# Fill Continuous Columns with Median Values
continuous_cols = ['roomcnt','taxvaluedollarcnt','taxamount','structuretaxvaluedollarcnt','calculatedfinishedsquarefeet','yearbuilt','landtaxvaluedollarcnt','calculatedbathnbr','lotsizesquarefeet','finishedsquarefeet12', 'unitcnt','fullbathcnt']

# Tract-Level Imputation
df_a['census_tract'] = df_a.censustractandblock.astype('string').apply(lambda x: x[:11]) # Create Tract String
tract_medians = df_a.loc[:, ['census_tract'] + continuous_cols].groupby('census_tract').filter(lambda x: len(x) >= 30).groupby('census_tract').median() # Minmum 30 Records per tract

# County Level Imputaiton
county_medians = df_a.loc[:, ['fips'] + continuous_cols].groupby('fips').median()

# Append Imputations to Dataframe
df_a_combined = pd.merge(pd.merge(df_a, tract_medians,'left', ['census_tract','census_tract'], suffixes = (None, '_tract')), county_medians, 'left', ['fips', 'fips'], suffixes = (None, '_county'))

for i in continuous_cols:
    
    state_median = df_a.loc[:,i].median()
    df_a.loc[:,i] = df_a.loc[:,i].fillna(df_a_combined.loc[:,f'{i}_tract'])  # Tract Level Imputaiton
    df_a.loc[:,i] = df_a.loc[:,i].fillna(df_a_combined.loc[:,f'{i}_county']) # County Level Imputation for Null-Valued Tracts
    df_a.loc[:,i] = df_a.loc[:,i].fillna(state_median)                       # State-Level Imputation for Null-Valued Counties
    
df_a.isna().sum().sort_values()/len(df_a)

parcelid                        0.000000
taxamount                       0.000000
landtaxvaluedollarcnt           0.000000
assessmentyear                  0.000000
taxvaluedollarcnt               0.000000
structuretaxvaluedollarcnt      0.000000
fireplaceflag                   0.000000
yearbuilt                       0.000000
unitcnt                         0.000000
roomcnt                         0.000000
propertylandusetypeid           0.000000
censustractandblock             0.000000
poolcnt                         0.000000
census_tract                    0.000000
hashottuborspa                  0.000000
fullbathcnt                     0.000000
fireplacecnt                    0.000000
fips                            0.000000
finishedsquarefeet12            0.000000
calculatedfinishedsquarefeet    0.000000
calculatedbathnbr               0.000000
bedroomcnt                      0.000000
bathroomcnt                     0.000000
lotsizesquarefeet               0.000000
propertycountyla

In [6]:
# # Fill Categorical Columns with Modal Values
# categorical_cols = ['propertycountylandusecode','regionidcity','propertyzoningdesc','buildingqualitytypeid','heatingorsystemtypeid']

# # Tract-Level Imputation
# tract_modes = df_a.loc[:, ['census_tract'] + categorical_cols].groupby('census_tract').filter(lambda x: len(x) >= 30).groupby('census_tract').agg(pd.Series.mode) # Minmum 30 Records per tract
# # County Level Imputaiton
# county_modes = df_a.loc[:, ['fips'] + categorical_cols].groupby('fips').agg(pd.Series.mode)

# # Append Imputations to Dataframe
# df_a_combined = pd.merge(pd.merge(df_a, tract_modes,'left', ['census_tract','census_tract'], suffixes = (None, '_tract')), county_modes, 'left', ['fips', 'fips'], suffixes = (None, '_county'))

# for i in categorical_cols:
    
#     state_mode = df_a.loc[:,i].value_counts().index[0]
#     df_a.loc[:,i] = df_a.loc[:,i].fillna(df_a_combined.loc[:,f'{i}_tract'])  # Tract Level Imputaiton
#     df_a.loc[:,i] = df_a.loc[:,i].fillna(df_a_combined.loc[:,f'{i}_county']) # County Level Imputation for Null-Valued Tracts
#     df_a.loc[:,i] = df_a.loc[:,i].fillna(state_mode)                         # State-Level Imputation for Null-Valued Counties
    

# df_a = df_a.dropna()
# df_a.isna().sum().sort_values()/len(df_a)

In [7]:
df_a.to_csv('2016_cleaned.csv')

In [8]:
# Columns by Null Records
df_b = df_b[df_b.censustractandblock.notna()] 

df_b['census_tract'] = df_b.censustractandblock.astype('string').apply(lambda x: x[:11]) # Create Tract String
df_b = df_b.loc[:, df_a.columns]


# Recode Features with Boolean Values
df_b.loc[:,bool_cols] = (df_b.loc[:,bool_cols] == True).astype(int)
    
# Code Zeros for "If Any" Features
df_b.loc[:, if_any_cols] = df_b.loc[:, if_any_cols].fillna(0) 

# # Append Median Imputations to Dataframe
df_b_combined = pd.merge(pd.merge(df_b, tract_medians,'left', ['census_tract','census_tract'], suffixes = (None, '_tract')), county_medians, 'left', ['fips', 'fips'], suffixes = (None, '_county'))

# # Apply Median Imputations
for i in continuous_cols:
    
    state_median = df_a.loc[:,i].median()
    df_b.loc[:,i] = df_b.loc[:,i].fillna(df_b_combined.loc[:,f'{i}_tract'])  # Tract Level Imputaiton
    df_b.loc[:,i] = df_b.loc[:,i].fillna(df_b_combined.loc[:,f'{i}_county']) # County Level Imputation for Null-Valued Tracts
    df_b.loc[:,i] = df_b.loc[:,i].fillna(state_median) 

# # # Append Mode to Dataframe
# df_b_combined = pd.merge(pd.merge(df_b, tract_modes,'left', ['census_tract','census_tract'], suffixes = (None, '_tract')), county_modes, 'left', ['fips', 'fips'], suffixes = (None, '_county'))

# for i in categorical_cols:
    
#     state_mode = df_a.loc[:,i].value_counts().index[0]
#     df_b.loc[:,i] = df_b.loc[:,i].fillna(df_b_combined.loc[:,f'{i}_tract'])  # Tract Level Imputaiton
#     df_b.loc[:,i] = df_b.loc[:,i].fillna(df_b_combined.loc[:,f'{i}_county']) # County Level Imputation for Null-Valued Tracts
#     df_b.loc[:,i] = df_b.loc[:,i].fillna(state_mode)                         # State-Level Imputation for Null-Valued Counties
    
    
df_b.isna().sum().sort_values()/len(df_b)

parcelid                        0.000000
taxamount                       0.000000
landtaxvaluedollarcnt           0.000000
assessmentyear                  0.000000
taxvaluedollarcnt               0.000000
structuretaxvaluedollarcnt      0.000000
fireplaceflag                   0.000000
yearbuilt                       0.000000
unitcnt                         0.000000
roomcnt                         0.000000
propertylandusetypeid           0.000000
censustractandblock             0.000000
poolcnt                         0.000000
census_tract                    0.000000
hashottuborspa                  0.000000
fullbathcnt                     0.000000
fireplacecnt                    0.000000
fips                            0.000000
finishedsquarefeet12            0.000000
calculatedfinishedsquarefeet    0.000000
calculatedbathnbr               0.000000
bedroomcnt                      0.000000
bathroomcnt                     0.000000
lotsizesquarefeet               0.000000
propertycountyla

In [9]:
df_b.to_csv('2017_cleaned.csv')